#### Extract wse variable from the SWOT LakeSP data.

In [11]:
import geopandas as gpd
from glob import glob
import numpy as np
import zipfile
import warnings
import pickle 
from scripts.date_transform import date_to_decimal
warnings.filterwarnings("ignore")


In [12]:
dir_lakeSP = 'data/swot_l2/lakeSP/erhai'
zip_files = glob(dir_lakeSP + '/*.zip')

Unzip the lakeSP file.

In [13]:
for file in zip_files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(file.split('.')[0])


Extract wse variable from the LakeSP data.

In [14]:
unzipped_files = sorted(glob(dir_lakeSP + '/*/*.shp'))
unzipped_files[0]


'data/swot_l2/lakeSP/erhai/SWOT_L2_HR_LakeSP_Prior_001_424_AS_20230805T082825_20230805T084112_PGC0_01/SWOT_L2_HR_LakeSP_Prior_001_424_AS_20230805T082825_20230805T084112_PGC0_01.shp'

In [15]:
lake_wse = {}
lake_name = 'ERH;ER HAI'   ## 'DIAN CHI', 'ERH;ER HAI'
# lake_name = 'DIAN CHI'   ## 'DIAN CHI', 'ERH;ER HAI'
for file in unzipped_files:
    print(file)
    file_gpd = gpd.read_file(file)
    ## select the features of target lake
    try: 
        ids = np.where(file_gpd['lake_name']==lake_name)[0] 
        id = np.argmax(file_gpd['geometry'][ids].area)
        lake = file_gpd.loc[ids[id]]
    except Exception as e:
        print(f"No data for lake")
    if lake['wse']  > -1000 and lake['area_total'] > 10 and lake['wse_std'] < 1:   ### filter out bad data
        date_decimal = date_to_decimal(lake['time_str'][:-1])
        date_decimal = round(date_decimal, 5)
        lake_wse[date_decimal] = lake['wse']
lake_wse


data/swot_l2/lakeSP/erhai/SWOT_L2_HR_LakeSP_Prior_001_424_AS_20230805T082825_20230805T084112_PGC0_01/SWOT_L2_HR_LakeSP_Prior_001_424_AS_20230805T082825_20230805T084112_PGC0_01.shp
data/swot_l2/lakeSP/erhai/SWOT_L2_HR_LakeSP_Prior_002_424_AS_20230826T051620_20230826T052318_PGC0_01/SWOT_L2_HR_LakeSP_Prior_002_424_AS_20230826T051620_20230826T052318_PGC0_01.shp
data/swot_l2/lakeSP/erhai/SWOT_L2_HR_LakeSP_Prior_003_077_AS_20230903T164304_20230903T165132_PGC0_01/SWOT_L2_HR_LakeSP_Prior_003_077_AS_20230903T164304_20230903T165132_PGC0_01.shp
data/swot_l2/lakeSP/erhai/SWOT_L2_HR_LakeSP_Prior_003_424_AS_20230916T015836_20230916T021123_PGC0_01/SWOT_L2_HR_LakeSP_Prior_003_424_AS_20230916T015836_20230916T021123_PGC0_01.shp
data/swot_l2/lakeSP/erhai/SWOT_L2_HR_LakeSP_Prior_004_424_AS_20231006T224337_20231006T225624_PGC0_01/SWOT_L2_HR_LakeSP_Prior_004_424_AS_20231006T224337_20231006T225624_PGC0_01.shp
data/swot_l2/lakeSP/erhai/SWOT_L2_HR_LakeSP_Prior_005_424_AS_20231027T192843_20231027T194130_PGC0_01

{2023.84464: 1966.471,
 2023.9018: 1966.438,
 2023.95897: 1966.371,
 2024.01609: 1966.227,
 2024.07309: 1966.582,
 2024.1301: 1966.234,
 2024.18711: 1965.97,
 2024.24411: 1965.902,
 2024.30112: 1965.78,
 2024.41514: 1965.599,
 2024.47214: 1965.743,
 2024.52915: 1965.557,
 2024.64316: 1965.931,
 2024.75718: 1966.425,
 2024.81419: 1966.009}

In [16]:
lake_wse


{2023.84464: 1966.471,
 2023.9018: 1966.438,
 2023.95897: 1966.371,
 2024.01609: 1966.227,
 2024.07309: 1966.582,
 2024.1301: 1966.234,
 2024.18711: 1965.97,
 2024.24411: 1965.902,
 2024.30112: 1965.78,
 2024.41514: 1965.599,
 2024.47214: 1965.743,
 2024.52915: 1965.557,
 2024.64316: 1965.931,
 2024.75718: 1966.425,
 2024.81419: 1966.009}

In [17]:
## write numpy data in 
path_save = dir_lakeSP+'/erhai_wse_lakesp.pkl'
with open(path_save, 'wb') as f:
        pickle.dump(lake_wse, f)


In [18]:
# with open(path_save, 'rb') as f:
#     heights_lake = pickle.load(f)
# heights_lake


Delete the unzipped folder.

In [19]:
import os
import shutil
unzipped_dir = [file for file in glob(dir_lakeSP + '/*') if not file.endswith('.zip')]
unzipped_dir
for folder in unzipped_dir:
    if os.path.isdir(folder):
        shutil.rmtree(folder)
        